In [1]:
import pandas as pd
import swifter
from multiprocessing import Pool

def function_to_apply(pandas):
    pandas["cost"] = pandas.apply(lambda x : x["Resource Quantity"]*x["Resource Unit Price"],axis=1, result_type='reduce')
    return pandas

def pandas_paralel_apply(full_df, num_proc, func):
    list_of_dfs = [full_df[full_df.index % num_proc==i] for i in range(0,num_proc)]
    with Pool(num_proc) as executor:
        df_new = pd.concat(executor.map(func, list_of_dfs))
    return df_new

def function_to_apply_big(pandas):
    pandas["cost"] = pandas.apply(lambda x : x["Project Cost"]*x["Project Cost"],axis=1, result_type='reduce')
    return pandas


In [2]:
def function_to_apply_row(x, y):
    return x*y

def function_to_apply_row_v1(x):
    return len(str(x).split(' '))
    

In [29]:
%timeit -n10 df = pd.read_csv('./data/Resources.csv', header=0, sep=',')
#8.15 s ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

8.15 s ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
df = pd.read_csv('./data/Resources.csv', header=0, sep=',')

In [9]:
%timeit -n1 -r7 df.to_csv('./output/output.csv')
#39.2 s ± 788 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

39.2 s ± 788 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
#Regular Pandas
%timeit -n1 -r7 df["cost"] = df["Resource Quantity"]*df["Resource Unit Price"]
#36.4 ms ± 7.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
%timeit -n1 -r7 df["cost"] = df.apply(lambda x : x["Resource Quantity"]*x["Resource Unit Price"], axis=1)
#2min 10s ± 766 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

%timeit -n1 -r7 df["cost"] = df.apply(lambda x : function_to_apply_row(x["Resource Quantity"], x["Resource Unit Price"]), axis=1)
#2min 10s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

36.4 ms ± 7.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2min 10s ± 766 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2min 10s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
#Swifter
%timeit -n1 -r7 df["cost"] = df.swifter.apply(lambda x : x["Resource Quantity"]*x["Resource Unit Price"], axis=1)
#40.5 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
%timeit -n1 -r7 df["cost"] = df.swifter.apply(lambda x : function_to_apply_row(x["Resource Quantity"], x["Resource Unit Price"]), axis=1)
#40 ms ± 708 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)

40.5 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
40 ms ± 708 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
#Multiprocessing
%timeit -n1 -r7 pandas_paralel_apply(df.copy(), 10, function_to_apply)
#46.9 s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

46.9 s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit -n1 -r7 agg_df = df.groupby("Resource Vendor Name").agg({"Resource Quantity" : 'mean'})
#429 ms ± 6.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
%timeit -n1 -r7 agg_df = df.fillna('0')
#3.36 s ± 18.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

429 ms ± 6.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.36 s ± 18.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
del df

NameError: name 'agg_df' is not defined

In [18]:
%timeit -n1 -r7 df = pd.read_csv('./data/resources_v2.csv', header=0, sep=',')
#31 s ± 75.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

31 s ± 75.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
df = pd.read_csv('./data/resources_v2.csv', header=0, sep=',')

In [20]:
%timeit -n1 -r7 df.to_csv('./output/output.csv')
#2min 41s ± 2.66 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

2min 41s ± 2.66 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
#Regular Pandas
%timeit -n1 -r7 df["cost"] = df["Resource Quantity"]*df["Resource Unit Price"]
#288 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
%timeit -n1 -r7 df["cost"] = df.apply(lambda x : x["Resource Quantity"]*x["Resource Unit Price"], axis=1)
#8min 18s ± 9.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
%timeit -n1 -r7 df["cost"] = df.apply(lambda x : function_to_apply_row(x["Resource Quantity"], x["Resource Unit Price"]), axis=1)
#8min 14s ± 1.25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

288 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
8min 18s ± 9.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
8min 14s ± 1.25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
#Swifter
%timeit -n1 -r7 df["cost"] = df.swifter.apply(lambda x : x["Resource Quantity"]*x["Resource Unit Price"], axis=1)
#272 ms ± 6.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
%timeit -n1 -r7 df["cost"] = df.swifter.apply(lambda x : function_to_apply_row(x["Resource Quantity"], x["Resource Unit Price"]), axis=1)
#274 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

272 ms ± 6.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
274 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit -n1 -r7 agg_df = df.groupby("Resource Vendor Name").agg({"Resource Quantity" : 'mean'})

%timeit -n1 -r7 agg_df = df.fillna('0')

1.9 s ± 20 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
14.5 s ± 626 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
#Multiprocessing
%timeit -n1 -r7 pandas_paralel_apply(df.copy(), 10, function_to_apply)

3min 29s ± 5.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [0]:
del df